## Introducción Scraping II


Trabajo de scraping sobre la web de FIFA para obtener información de los jugadores de la selección española de fútbol en el Mundial de 2022

> Automatización de la descarga del archivo html



In [ ]:
! pip install --upgrade pip
! pip install selenium

## Instalación del driver

https://selenium-python.readthedocs.io/installation.html#drivers

Tienes que descargar el driver del navegador que vayas a usar.

### Driver de Firefox
Descarga la versión adecuada a tu navegador (versión) y sistema operativo de https://github.com/mozilla/geckodriver/releases

### Driver de Chrome    
https://sites.google.com/chromium.org/driver/downloads

Descomprime el driver en esta carpeta.

In [ ]:
# Ejemplo usando Firefox
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

# Options para headless (sin interfaz gráfica)
options = Options()
options.add_argument('-headless')

url = 'https://www.fifa.com/fifaplus/es/tournaments/mens/worldcup/qatar2022/teams/spain/squad'

driver = webdriver.Firefox(options = options)

# si no encuentra el binary
# driver = webdriver.Firefox(firefox_binary = '/usr/bin/firefox', options = options)

driver.get(url)
driver.implicitly_wait(10) # para carga de js?


In [ ]:
with open('spain.html', 'w') as f:
    f.write(driver.page_source)

¿Ahora podrías usar la web de los equipos para descargarlos todos? 

https://www.fifa.com/fifaplus/es/tournaments/mens/worldcup/qatar2022/teams

In [ ]:
# Cargar la página de equipos con el navegador
url = 'https://www.fifa.com/fifaplus/es/tournaments/mens/worldcup/qatar2022/teams'
driver.get(url)
driver.implicitly_wait(10) # para carga de js?

In [ ]:
# guardo la página web con los equipos

with open('equipos.html', 'w') as f:
    f.write(driver.page_source)

In [ ]:
# Cargamos nuevo archivo equipos.html
from bs4 import BeautifulSoup

html_doc = open('equipos.html').read()

soup = BeautifulSoup(html_doc, 'html.parser')

In [ ]:
# Buscamos los equipos
equipos = soup.select('div[class*="flag-grid_flagGrid"] a')
print(len(equipos))


In [ ]:
# Preparamos función para extraer los datos de cada país

def datos_pais(p):
    nombre = p.select('div[class*="flag-with-info_flagCountry"]')[0].text
    abr = p.select('div[class*="flag-with-info_flagAbbr"]')[0].text.replace('(', '').replace(')', '')
    url = f'https://www.fifa.com{p.attrs["href"]}'.replace('news', 'squad')  # url de la plantilla
    bandera = p.select('div[class*="flag_flagLarge"] img')[0]['src']
    return {'nombre': nombre, 'abr': abr, 'url': url, 'bandera': bandera}

In [ ]:
datos = [datos_pais(p) for p in equipos]

In [ ]:
import json

json.dump(datos, open('equipos.json', 'w'), indent = 4)


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

for p in datos:
    url = p['url']
    driver.get(url)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH,
         '//div[contains(@class, "manager-badge-card_badgeCard")]')))

    with open(f'equipos/{p["nombre"]}.html', 'w') as f:
        f.write(driver.page_source)
    time.sleep(3) # para no saturar el servidor
    